# COBRApy

COBRApy is a package for constraint-based modeling of biological networks written in Python.

This tool allows loading and inspecting Genome-Scale Metabolic (GEM) models written in the Sytems Biology Markup Language (SBML) format.

Using COBRApy, one can analyse the following model contents:
1. Reactions
2. Metabolites
3. Genes
4. Exchange reactions (Environmental Conditions)

COBRApy allows manipulating the contents of a GEM model. For instance, one can edit reactions' flux bounds, knock out a metabolic gene, or change the environmental conditions.

Phenotype prediction can be simulated with several flux analysis methods implemented in COBRApy. These include Flux Balance Anlysis (FBA), Parsimonious FBA, or Flux Variability Analysis (FVA).

The simulation of gene and reaction deletions for a given GEM model is a simple and straightforward process. One can simulate single or double knock outs using one of the flux analysis methods mentioned above.

## Instalation


### Requirements
The following requirements need to be installed to use COBRApy:
- Python 3.6 or higher
- pip must be installed
- GLPK solver is used by default but CPLEX is prefered


### How to install COBRApy?
```
pip install cobra
```

# Exercise 5 - Phenotype prediction

## Working with a GEM model

For this practical session, we will be using the following model:
- _E. coli_ core model which contains the central carbon metabolism of _Escherichia coli_ -> file: e_coli_core.xml

This exercise consists of exploring the phenotype prediction tools of COBRApy. Thus, the following steps will be followed:
- Loading the model with COBRApy;
- Perform a FBA simulation using an aerobic/anaerobic conditions;
- Perform reaction and gene deletions;
- Perform other flux analysis methods, such as pFBA, FVA, ROOM and MOMA;
- Analyzing the model essential reactions and genes;

In [ ]:
# importing cobra
import cobra

# Loading a model
model_path = '../data/e_coli_core.xml'
model = cobra.io.read_sbml_model(model_path)

model

### Phenotype Prediction

COBRApy includes three different algortithms for phenotype prediction. These include:
- Flux Balance Analysis (FBA); 
- Parsimonious Flux Balance Analysis (pFBA);
- Flux Variability Analysis (FVA);
- Regulatory On/Off Minimization (ROOM);
- Minimization of Metabolic Adjustment (MOMA);

#### Flux Balance Analysis (FBA) - Aerobiosis

First, we should verify our default environmental conditions to check the aerobic conditions.

In [ ]:
#inspecting the exchange reactions.
for exchange in model.exchanges:
    print(exchange.name, '->', exchange.bounds)

In [ ]:
# performing a FBA simulation in aerobiosis
model.summary()

#### Flux Balance Analysis (FBA) - Anaerobiosis

Now, we should change the model exchange reactions to anaerobic conditions.

By default, any alteration performed in a GEM model using CORBApy is irreversible. 
That is, if we change the bounds of the oxygene exchange reaction, our model will 

In [ ]:
#setting an anaerobic medium
o2 = model.exchanges.get_by_id('EX_o2_e')
o2.bounds = (0,1000)
o2

In [ ]:
#single reaction deletion
reaction_deletion_results = cobra.flux_analysis.single_reaction_deletion(model)
reaction_deletion_results

In [ ]:
#single gene deletion
gene_deletion_results = cobra.flux_analysis.single_gene_deletion(model)
gene_deletion_results

It is worth noting that genes and reactions with a growth value equal to zero can be considered as essential genes or essential reactions, respectively.

#### Double Delections

Double gene and reaction delections can also be simulated with the flux analysis package of COBRApy. To do so the `cobra.flux_analysis.double_gene_deletion()` and `cobra.flux_analysis.double_reaction_deletion()` methods. These methods test the deletion of all the possible combinations.

In [ ]:
#double reaction deletion
double_reaction_deletion_results = cobra.flux_analysis.double_reaction_deletion(model)
double_reaction_deletion_results

In [ ]:
#double gene deletion
double_gene_deletion_results = cobra.flux_analysis.double_gene_deletion(model)
double_gene_deletion_results

### Production envelopes

Production Envelopes can be used to show distinct phases of optimal growth using two different carbon substrates. If we want to access a phenotype phase plane to evaluate uptakes of Glucose and Oxygen:

In [ ]:
#How to perform a production envolope
prod_env = cobra.flux_analysis.production_envelope(model, ["EX_glc__D_e", "EX_o2_e"])
prod_env.head()

Moreover, if we want to specify the carbon source, the `production_envelope()` can also return the carbon and mass yield. For instance, when the objective is to produce Acetate, it is possible to obtain a production envelope as follows and quickly plot the results:

In [ ]:
#obtain a plot of a production envelope using glucose as carbon source and optimizing Acetate production
prod_env = cobra.flux_analysis.production_envelope(model, ["EX_o2_e"], objective="EX_ac_e", carbon_sources="EX_glc__D_e")
prod_env.plot(kind='line', x='EX_o2_e', y='carbon_yield_maximum')

Finally, the production envelope can also be used obtain a flux variation plot. This can be achieved as follows:

In [ ]:
#obtain a flux varariation plot
prod_env_co2 = cobra.flux_analysis.production_envelope(model, ["Biomass_Ecoli_core"], objective=['EX_co2_e'])
prod_env_co2.plot(kind='line', x='Biomass_Ecoli_core', y=['flux_minimum', 'flux_maximum'])

### Pathway Visualization

COBRApy does not include any pathway visualization method. Nevertheless, independent tools that work with cobra models can be use for that goal. One of such tools is named Escher.

Escher allows the visualization of metabolic pathway maps. Escher maps can be build using the `escher.Build()` function, which requires the model and a FBA solution.

To obtain the metabolic map for the model in question, Escher can be run as follows:

In [ ]:
import escher

builder = escher.Builder(map_name='e_coli_core.Core metabolism', model=model, reaction_data=fba_solution.fluxes)
builder